## Tutorial

### Saving Models with BentoML

To begin with BentoML, you will need to save your trained models with BentoML API in its model store (a local directory managed by BentoML). The model store is used for managing all your trained models locally as well as accessing them for serving

In [1]:
import bentoml

from sklearn import svm
from sklearn import datasets

In [2]:
# load training data set
iris = datasets.load_iris()
x, y = iris.data, iris.target

# train the model
clf = svm.SVC(gamma='scale')
clf.fit(x, y)

# save the model to the BentoML local model store
saved_model = bentoml.sklearn.save_model('iris_clf', clf)
print(f"Model saved: {saved_model}")

Model saved: Model(tag="iris_clf:qfy5hzsrc6g5c6cp")


The model is now saved under the name iris_clf with an automatically generated version. The name and version pair can then be used for retrieving the model. For instance, the original model object can be loaded back into memory for testing via:

In [3]:
model = bentoml.sklearn.load_model("iris_clf:qfy5hzsrc6g5c6cp")

# alternatively, use 'latest' to find the newest version
model = bentoml.sklearn.load_model('iris_clf:latest')

The bentoml.sklearn.save_model API is built specifically for the Scikit-Learn framework and uses its native saved model format under the hood for best compatibility and performance. 

### Creating a Service
Services are the core components of BentoML, where the serving logic is defined. Create a file service.py with:

In [5]:
import numpy as np
import bentoml
from bentoml.io import NumpyNdarray

In [7]:
iris_clf_runner = bentoml.sklearn.get('iris_clf:latest').to_runner()

service = bentoml.Service('iris_classifier', runners=[iris_clf_runner])

@service.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_series: np.ndarray) -> np.ndarray:
    result = iris_clf_runner.predict.run(input_series)
    
    return result

Send prediction request to the service:

In [ ]:
import requests

requests.post(
   "http://127.0.0.1:3000/classify",
   headers={"content-type": "application/json"},
   data="[[5.9, 3, 5.1, 1.8]]",
).text